In [24]:
import os
import json
import pandas as pd

def collect_data(top_folder, ctx_size):
    """ Collects the minimum test loss and corresponding parameters across all subfolders in the main folder. """
    results = []
    # Traverse through each subfolder in the main folder
    for submain_folder in os.listdir(top_folder):
        main_folder = os.path.join(top_folder, submain_folder)
        for subfolder in os.listdir(main_folder):
            subfolder_path = os.path.join(main_folder, subfolder)
            progress_file = os.path.join(subfolder_path, 'progress.csv')
            params_file = os.path.join(subfolder_path, 'params.json')
            
            # Check if both necessary files exist
            if os.path.exists(progress_file) and os.path.exists(params_file):
                try:
                    # Read progress.csv and find the minimum test loss
                    data = pd.read_csv(progress_file)
                    data.fillna(0, inplace=True)
                    # Read params.json
                    with open(params_file, 'r') as file:
                        params = json.load(file)

                        def check_ctx_size(ctx_size):
                            if ctx_size == None:
                                return True
                            if 'context' in params and params['context'] == ctx_size:
                                return True
                            if 'for_all_networks' in params and params['for_all_networks'] == ctx_size:
                                return True
                            return False
                        if check_ctx_size(ctx_size) == False:
                            continue
                        # Collect required params and the corresponding test loss
                        result = {}
                        param_dict = {
                            'n_stores': 'n_stores',
                            'context': 'context',
                            'learning_rate': 'learning_rate',
                            'master_neurons': 'master_neurons',
                            'store_embedding': 'store_embedding',
                            'for_all_networks': 'for_all_networks'
                        }
                        for key, value in param_dict.items():
                            if value in params:
                                result[key] = params.get(value)
                        result['best_dev_loss'] = data['dev_loss'].min()
                        result['test_loss(at best_dev)'] = data[data['dev_loss'] == result['best_dev_loss']]['test_loss'].iloc[0]
                        result['train_loss(at best_dev)'] = data[data['dev_loss'] == result['best_dev_loss']]['train_loss'].iloc[0]
                        result['best_test_loss'] = data['test_loss'].min()
                        result['best_train_loss'] = data['train_loss'].min()
                        result['path'] = subfolder_path
                        results.append(result)
                    # if 'test_loss' in data.columns:
                    #     min_loss = data['test_loss'].min()
                    #     # Read params.json
                    #     with open(params_file, 'r') as file:
                    #         params = json.load(file)
                    #         context_size = params.get('context_size')
                    #         # Collect required params and the corresponding test loss
                    #         result = {
                    #             'learning_rate': params.get('learning_rate'),
                    #             'best_test_loss': min_loss
                    #         }
                    #         results.append(result)
                except Exception as e:
                    print(f"Error processing files in {subfolder_path}: {e}")

    return results

def create_results_table(main_folder, ctx_size):
    """ Creates a table of the minimum test losses for each combination of learning_rate, context_size, and samples. """
    data = collect_data(main_folder, ctx_size)
    if data:
        df = pd.DataFrame(data)
        return df.sort_values(by='best_dev_loss', ascending=True)
    return None

def make_the_result_table(paths, context_sizes):
    # Placeholder for results
    results = []

    # Iterate over each path and context size, call the function, and store the top row
    for num_stores, path in paths.items():
        for context_size in context_sizes:
            df = create_results_table(path, context_size)
            if df is None:
                continue
            top_row = df.iloc[0]  # Get the top row
            # Extract necessary columns and add custom columns
            result_row = {
                "# of stores": num_stores,
                "context size": context_size,
                "Learning Rate": top_row['learning_rate'],
                "Train Loss": top_row['train_loss(at best_dev)'],
                "Dev Loss": top_row['best_dev_loss'],
                "Test Loss": top_row['test_loss(at best_dev)'],
                # "path": top_row['path']
            }
            results.append(result_row)

    # Combine all top rows into a single DataFrame
    return pd.DataFrame(results)

Synthetic data, one warehouse lost demand

In [21]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/50"
}

df = make_the_result_table(paths, [0, 1, 256])
print(df.to_string(index=False))

 # of stores  context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3             0         0.0100    5.670470  5.657994   5.663223
           3             1         0.0100    5.613812  5.608477   5.609850
           3           256         0.0100    5.616237  5.611364   5.612627
           5             0         0.0010    5.312334  5.292432   5.301958
           5             1         0.0100    5.256996  5.235224   5.247245
           5           256         0.0001    5.249202  5.237397   5.249729
          10             0         0.0100    5.773343  5.808772   5.776743
          10             1         0.0100    5.712263  5.755829   5.720371
          10           256         0.0100    5.731197  5.761904   5.729029
          20             0         0.0010    5.902905  5.892253   5.871639
          20             1         0.0010    5.850531  5.839253   5.818287
          20           256         0.0100    5.857646  5.844117   5.820630
          30             

In [25]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/50"
}

df = make_the_result_table(paths, [1, 2, 4, 8, 16, 32, 64, 128])
#df = make_the_result_table(paths, [1, 32, 256])
print(df.to_string(index=False))

 # of stores  context size  Learning Rate  Train Loss  Dev Loss   Test Loss
           3             1           0.01    5.673688  5.659562    5.665522
           3             2           0.01    5.682550  5.663403    5.671105
           3             4           0.01    5.619100  5.612201    5.613837
           3             8           0.01    5.621052  5.612698    5.615135
           5             1           0.01    5.317029  5.297034    5.303407
           5             2           0.01    5.254412  5.235755    5.246651
          10             1           0.01   16.467711 16.325007   16.340109
          10             2           0.01    9.930729  9.974490  320.575352
          10             4           0.01   11.576283 11.530542   12.204801
          10             8           0.01    5.789521  5.815527    5.785925
          10            16           0.01    5.725805  5.768559    5.731618
          20             1           0.01   20.530435 20.416127   19.890959
          20

In [26]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/50"
}

df = make_the_result_table(paths, [1, 2, 4, 8, 16, 32, 64, 128])
#df = make_the_result_table(paths, [1, 32, 256])
print(df.to_string(index=False))

 # of stores  context size  Learning Rate  Train Loss   Dev Loss   Test Loss
           3             1          0.010    5.630121   5.619005    5.624080
           3             2          0.010    5.629419   5.613049    5.615874
           3             4          0.010    5.620411   5.610767    5.611920
           3             8          0.010    5.619027   5.609878    5.610715
           5             1          0.010    5.270876   5.247549    5.257497
           5             2          0.010    5.335778   5.302212    5.309347
           5             4          0.010    5.258151   5.236567    5.248581
           5             8          0.010    5.262214   5.241388    5.251634
          10             1          0.010    5.783530   5.812656    5.778496
          10             2          0.010    7.789676   7.823373    7.801336
          10             4          0.010    5.725203   5.762549    5.726730
          10             8          0.010    6.771707   6.801686    6.771661

In [34]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/50"
}
df_ctx = make_the_result_table(paths, [0, 1, 256])
df_ctx.insert(1, 'Architecture Class', "Symmetry_Aware")
df_ctx = df_ctx.loc[df_ctx.groupby(['# of stores'])['Dev Loss'].idxmin()]

paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/50"
}
df_gnn = make_the_result_table(paths, [1, 2, 4, 8, 16, 32, 64, 128])
df_gnn.insert(1, 'Architecture Class', "GNN")
df_gnn = df_gnn.loc[df_gnn.groupby(['# of stores'])['Dev Loss'].idxmin()]

paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/3",
    5: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/5",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/20",
    30: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/30",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/50"
}
df_gnn_mp = make_the_result_table(paths, [1, 2, 4, 8, 16, 32, 64, 128])
df_gnn_mp.insert(1, 'Architecture Class', "GNN Message Passing")
df_gnn_mp = df_gnn_mp.loc[df_gnn_mp.groupby(['# of stores'])['Dev Loss'].idxmin()]

vanilla = [
    {
                "# of stores": 3,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.610,
                "Dev Loss": 5.610,
                "Test Loss": 5.610,
            },
    {
                "# of stores": 5,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.250,
                "Dev Loss": 5.250,
                "Test Loss": 5.240,
            },
    {
                "# of stores": 10,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.720,
                "Dev Loss": 5.740,
                "Test Loss": 5.720,
            },
    {
                "# of stores": 20,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.850,
                "Dev Loss": 5.870,
                "Test Loss": 5.850,
            },
    {
                "# of stores": 30,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.580,
                "Dev Loss": 5.60,
                "Test Loss": 5.59,
            },
    {
                "# of stores": 50,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.410,
                "Dev Loss": 5.400,
                "Test Loss": 5.420,
            },
]
df_vanilla = pd.DataFrame(vanilla)

df = pd.concat([df_ctx, df_gnn, df_gnn_mp, df_vanilla])
architecture_order = ['Symmetry_Aware', 'GNN', 'GNN Message Passing', 'Vanilla']
df['Architecture Class'] = pd.Categorical(df['Architecture Class'], categories=architecture_order, ordered=True)
df.sort_values(by=['# of stores', 'Architecture Class'], inplace=True)
print(df.to_string(index=False))

 # of stores  Architecture Class context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3      Symmetry_Aware            1          0.010    5.613812  5.608477   5.609850
           3                 GNN            4          0.010    5.619100  5.612201   5.613837
           3 GNN Message Passing            8          0.010    5.619027  5.609878   5.610715
           3             Vanilla         None            NaN    5.610000  5.610000   5.610000
           5      Symmetry_Aware            1          0.010    5.256996  5.235224   5.247245
           5                 GNN            2          0.010    5.254412  5.235755   5.246651
           5 GNN Message Passing            4          0.010    5.261039  5.235965   5.246145
           5             Vanilla         None            NaN    5.250000  5.250000   5.240000
          10      Symmetry_Aware            1          0.010    5.712263  5.755829   5.720371
          10                 GNN           16          0.010

Synthetic data, Transshipment

In [19]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/transshipment/3",
    5: "/user/ml4723/Prj/NIC/ray_results/transshipment/5",
    10: "/user/ml4723/Prj/NIC/ray_results/transshipment/10",
}
df_sym = make_the_result_table(paths, [0, 1, 256])
df_sym.insert(1, 'Architecture Class', "Symmetry_Aware")

paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/3",
    5: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/5",
    10: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/10",
}
df_van = make_the_result_table(paths, [None])
df_van.insert(1, 'Architecture Class', "Vanilla")

lower_bound = [
    {
                "# of stores": 3,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 6.19,
            },
    {
                "# of stores": 5,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 5.75,
            },
    {
                "# of stores": 10,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 6.05,
            },
]
df_lower_bound = pd.DataFrame(lower_bound)

df = pd.concat([df_sym, df_van, df_lower_bound])
df.sort_values(by=['# of stores', 'context size'], inplace=True)
print(df.to_string(index=False))

 # of stores Architecture Class context size  Learning Rate  Train Loss   Dev Loss  Test Loss
           3     Symmetry_Aware            0         0.0100   15.786119  15.745235  38.493707
           3     Symmetry_Aware            1         0.0010    6.451584   6.442219   6.436544
           3     Symmetry_Aware          256         0.0100    6.215819   6.195688   6.190809
           3            Vanilla         None         0.0001    6.199735   6.195936   6.191248
           3        Lower bound         None            NaN         NaN        NaN   6.190000
           5     Symmetry_Aware            0         0.0010   14.135803  14.153102  34.991883
           5     Symmetry_Aware            1         0.0100    6.061970   6.045415   6.036085
           5     Symmetry_Aware          256         0.0010    5.751615   5.759176   5.751703
           5            Vanilla         None         0.0001    5.755991   5.759228   5.751869
           5        Lower bound         None            NaN 